In [12]:
from selenium import webdriver
import time
import pandas as pd
import os

from selenium.webdriver.support.select import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [13]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

# Replace with the actual path to your chromedriver.exe
chromedriver_path = 'chromedriver.exe'

service = Service(chromedriver_path)
driver = webdriver.Chrome(service=service)

url = 'https://www.hondacengkareng.com/kategori-produk/suku-cadang-resmi-motor-honda/key-set-kunci-kontak-resmi-motor-honda/'

driver.get(url)
driver.implicitly_wait(10)

In [14]:
# Daftar untuk menyimpan detail produk
nama_produk = []
harga = []
kode_produk = []
kategori = []
motor_implementasi = []
berat = []
warna = []
dimensi = []

In [15]:
nama_produk.clear()
harga.clear()
kode_produk.clear()
kategori.clear()
motor_implementasi.clear()
berat.clear()
warna.clear()
dimensi.clear()

In [16]:
def click_all_product_links_on_page():
    try:
        # Tunggu elemen produk muncul
        links = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, 'woocommerce-loop-product__title'))
        )
    except Exception as e:
        print(f"Error occurred while finding product links: {e}")
        return

    # Cek apakah ada link yang ditemukan
    if not links:
        print("Tidak ada produk yang ditemukan di halaman ini.")
        return

    for i in range(len(links)):
        try:
            # Ambil elemen kembali setiap iterasi untuk memastikan DOM terbaru
            links = driver.find_elements(By.CLASS_NAME, 'woocommerce-loop-product__title')
            if i >= len(links):
                print(f"Indeks {i} tidak ditemukan dalam daftar links.")
                continue

            link = links[i]
            link.click()

            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, 'product_title.entry-title'))
            )

            # Mengambil informasi produk
            try:
                title = driver.find_element(By.CLASS_NAME, 'product_title.entry-title')
                product_name = title.text
                nama_produk.append(product_name)
            except Exception as e:
                print(f"Error occurred while retrieving product name: {e}")
                nama_produk.append('')

            try:
                # Mengambil harga produk
                amount = driver.find_element(By.CLASS_NAME, 'woocommerce-Price-amount')
                harga_produk = amount.text
                harga.append(harga_produk)
            except Exception as e:
                print(f"Error occurred while retrieving product price: {e}")
                harga.append('')

            try:
                # Mengambil kode produk
                sku = driver.find_element(By.CSS_SELECTOR, 'span.sku')
                kode_produk.append(sku.text)
            except Exception as e:
                print(f"Error occurred while retrieving product code: {e}")
                kode_produk.append('')

            try:
                # Mengambil kategori produk
                category = driver.find_element(By.CSS_SELECTOR, 'span.posted_in')
                kategori.append(category.text)
            except Exception as e:
                print(f"Error occurred while retrieving product category: {e}")
                kategori.append('')

            try:
                # Mengambil model motor yang cocok
                motor_section = driver.find_element(By.CSS_SELECTOR, 'section.honda-bike-taxonomy')
                motor_implementasi.append(motor_section.text)
            except Exception as e:
                print(f"Error occurred while retrieving motor implementation: {e}")
                motor_implementasi.append('')

            try:
                # Klik untuk menampilkan informasi tambahan
                additional_info_tab = driver.find_element(By.ID, 'tab-title-additional_information')
                additional_info_tab.click()
                WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CLASS_NAME, 'product_weight'))
                )

                # Mengambil berat produk
                weight_element = driver.find_element(By.CLASS_NAME, 'product_weight')
                berat_produk = weight_element.text.split(' ')[0] + ' ' + weight_element.text.split(' ')[1]
                berat.append(berat_produk)
            except Exception as e:
                print(f"Error occurred while retrieving product weight: {e}")
                berat.append('')

            try:
                # Mengambil warna produk
                color_element = driver.find_element(By.XPATH, '//th[text()="Warna"]/following-sibling::td/p')
                warna_produk = color_element.text
                warna.append(warna_produk)
            except Exception as e:
                print(f"Error occurred while retrieving product color: {e}")
                warna.append('')
            
            try:
                # Mengambil dimensi produk
                dimension_element = driver.find_element(By.CLASS_NAME, 'product_dimensions')
                dimensi_produk = dimension_element.text
                dimensi.append(dimensi_produk)
            except Exception as e:
                print(f"Error occurred while retrieving product dimension: {e}")
                dimensi.append('')

            # Cetak detail produk untuk verifikasi
            print("Nama produk:", nama_produk[-1])
            print("Harga produk:", harga[-1])
            print("Kode produk:", kode_produk[-1])
            print("Kategori produk:", kategori[-1])
            print("Motor implementasi:", motor_implementasi[-1])
            print("Berat produk:", berat[-1])
            print("Warna produk:", warna[-1])
            print("Dimensi produk:", dimensi[-1])
            
            # Kembali ke halaman sebelumnya
            driver.back()
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, 'woocommerce-loop-product__title'))
            )
            
        except Exception as e:
            print(f"Error occurred while clicking or retrieving product information: {e}")

# Ulangi untuk setiap halaman
total_pages = 4  # Ubah ke jumlah total halaman
for page_num in range(1, total_pages + 1):
    print(f"Processing page {page_num}")
    
    click_all_product_links_on_page()
    
    # Navigasi ke halaman berikutnya jika ada
    if page_num < total_pages:
        try:
            next_page_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.LINK_TEXT, str(page_num + 1)))
            )
            next_page_button.click()
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, 'woocommerce-loop-product__title'))
            )
        except Exception as e:
            print(f"Could not navigate to the next page: {e}")

# Cetak semua detail produk yang dikumpulkan di akhir
print("All collected product names:", nama_produk)
print("All collected prices:", harga)
print("All collected product codes:", kode_produk)
print("All collected categories:", kategori)
print("All collected motor implementations:", motor_implementasi)
print("All collected weights:", berat)
print("All collected colors:", warna)
print("All collected dimensions:", dimensi)

Processing page 1
Error occurred while retrieving product dimension: Message: no such element: Unable to locate element: {"method":"css selector","selector":".product_dimensions"}
  (Session info: chrome=126.0.6478.183); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7A4893E52+31618]
	(No symbol) [0x00007FF7A480B0B9]
	(No symbol) [0x00007FF7A46C888A]
	(No symbol) [0x00007FF7A4718524]
	(No symbol) [0x00007FF7A471862C]
	(No symbol) [0x00007FF7A475F787]
	(No symbol) [0x00007FF7A473D14F]
	(No symbol) [0x00007FF7A475CA80]
	(No symbol) [0x00007FF7A473CEB3]
	(No symbol) [0x00007FF7A470A46B]
	(No symbol) [0x00007FF7A470B001]
	GetHandleVerifier [0x00007FF7A4B9A02D+3202397]
	GetHandleVerifier [0x00007FF7A4BE6A4D+3516285]
	GetHandleVerifier [0x00007FF7A4BDC4C0+3473904]
	GetHandleVerifier [0x00007FF7A4945D56+760454]
	(No symbol) [0x00007FF7A4816B5F]
	(No symbol

In [17]:
df_nama_produk = pd.DataFrame(nama_produk, columns=["Nama Produk"])
df_harga = pd.DataFrame(harga, columns=["Harga"])
df_kode_produk = pd.DataFrame(kode_produk, columns=["Kode Produk"])
df_kategori = pd.DataFrame(kategori, columns=["Kategori"])
df_motor_implementasi = pd.DataFrame(motor_implementasi, columns=["Motor Implementasi"])
df_berat = pd.DataFrame(berat, columns=["Berat"])
df_warna = pd.DataFrame(warna, columns=["Warna"])
df_dimensi = pd.DataFrame(dimensi, columns=["Dimensi"])

# Combine all DataFrames into a single DataFrame
final_df = pd.concat([
    df_nama_produk,
    df_harga,
    df_kode_produk,
    df_kategori,
    df_motor_implementasi,
    df_berat,
    df_warna,
    df_dimensi
], axis=1)

print(final_df)

                                           Nama Produk         Harga  \
0          Kunci Jok (Emergency Key) Honda PCX 150 K97     Rp 44,000   
1                 Remot (Fob) Honda New Vario 150 K59J    Rp 305,000   
2                 Remot (Fob) Honda New Vario 150 K59J    Rp 332,000   
3     Kunci Kontak (Key Set) Honda BeAT Sporty eSP K81    Rp 245,500   
4               Bahan Kunci (Key Blank) Honda CRF 150L     Rp 39,000   
..                                                 ...           ...   
167  Kunci Kontak (Switch Handel Lock) Honda Scoopy...    Rp 396,000   
168  Kunci Kontak (Key set) Honda New Vario 125 eSP...    Rp 282,000   
169              SCU, Smart Control Unit Honda ADV 150  Rp 1,376,000   
170                    Kunci Kontak Honda BeAT eSP K81    Rp 113,500   
171                    Kunci Kontak Honda BeAT eSP K81    Rp 184,500   

     Kode Produk                                  Kategori  \
0    35194K35V31          Kategori: Key Set (Kunci Kontak)   
1    35121K

In [18]:
import pandas as pd

# Save the DataFrame to a CSV file
final_df.to_csv('emblem_motor_honda.csv', index=False, sep=';', encoding='utf-8')

print("DataFrame has been saved to 'key_set_motor_honda.csv'")

DataFrame has been saved to 'key_set_motor_honda.csv'


In [19]:
from IPython.display import FileLink

# Generate a download link
FileLink('key_set_motor_honda.csv')

c:\Users\basti\OneDrive\Documents\ds_salary_proj\key_set_motor_honda.csv

In [20]:
import csv

# Path to save the CSV file
file_path = 'key_set_motor_honda.csv'

# Data headers
headers = ['Nama Produk', 'Harga', 'Kode Produk', 'Kategori', 'Motor Implementasi', 'Berat', 'Warna', 'Dimensi']

# Data rows
rows = zip(nama_produk, harga, kode_produk, kategori, motor_implementasi, berat, warna, dimensi)

# Write the CSV file with UTF-8 BOM encoding
with open(file_path, mode='w', encoding='utf-8-sig', newline='') as file:
    writer = csv.writer(file, delimiter=';')
    
    # Write the headers
    writer.writerow(headers)
    
    # Write the rows
    writer.writerows(rows)

print(f"Data successfully written to {file_path}")

# Generate a download link
display(FileLink(file_path))

Data successfully written to key_set_motor_honda.csv


c:\Users\basti\OneDrive\Documents\ds_salary_proj\key_set_motor_honda.csv